# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
!pip show hvplot

Name: hvplot
Version: 0.9.2
Summary: A high-level plotting API for the PyData ecosystem built on HoloViews.
Home-page: https://hvplot.holoviz.org
Author: Philipp Rudiger
Author-email: developers@pyviz.org
License: BSD
Location: C:\Users\HP\anaconda3\Lib\site-packages
Requires: bokeh, colorcet, holoviews, numpy, packaging, pandas, panel, param
Required-by: 


In [2]:
!pip show geoviews

Name: geoviews
Version: 1.11.1
Summary: GeoViews is a Python library that makes it easy to explore and visualize geographical, meteorological, and oceanographic datasets, such as those used in weather, climate, and remote sensing research.
Home-page: https://geoviews.org
Author: 
Author-email: 
License: BSD 3-Clause
Location: C:\Users\HP\anaconda3\Lib\site-packages
Requires: bokeh, cartopy, holoviews, numpy, packaging, panel, param, pyproj, shapely, xyzservices
Required-by: 


In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,280.20,65,0,3.09,US,1712454404
1,1,hamilton,39.1834,-84.5333,283.62,63,0,2.57,US,1712454389
2,2,college,64.8569,-147.8028,277.64,42,75,0.00,US,1712454573
3,3,melekeok village,7.4957,134.6367,303.46,70,75,5.14,PW,1712454722
4,4,grytviken,-54.2811,-36.5092,277.12,95,100,5.62,GS,1712454568


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    size = "Humidity",
    hover_color = ["City"]
)

# Display the map
cities_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
city_data_df["Max Temp"].max()

308.14

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
cities_filtered = pd.DataFrame(city_data_df.loc[(city_data_df["Wind Speed"]<4.5) & (city_data_df["Max Temp"]<300)])

# Drop any rows with null values
cities_filtered = cities_filtered.dropna()

# Display sample data
cities_filtered.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,280.20,65,0,3.09,US,1712454404
1,1,hamilton,39.1834,-84.5333,283.62,63,0,2.57,US,1712454389
2,2,college,64.8569,-147.8028,277.64,42,75,0.00,US,1712454573
5,5,labytnangi,66.6572,66.4183,263.92,86,100,4.04,RU,1712454722
6,6,waitangi,-43.9535,-176.5597,286.94,75,93,1.34,NZ,1712454568


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
cities_filtered[cities_filtered.City == "waitangi"]

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,waitangi,-43.9535,-176.5597,286.94,75,93,1.34,NZ,1712454568


In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cities_filtered[["City","Country", "Lat","Lng","Humidity"]]
# hotel_df = hotel_df.T.set_index("City")
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# hotel_df["City"]=hotel_df.index

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bethel,US,41.3712,-73.4140,65,
1,hamilton,US,39.1834,-84.5333,63,
2,college,US,64.8569,-147.8028,42,
5,labytnangi,RU,66.6572,66.4183,86,
6,waitangi,NZ,-43.9535,-176.5597,75,


In [11]:
geoapify_key

'ac13b42866534833b92ca5b1702408e7'

In [13]:
params = {
    "categories":"accomodation.hotel",
    "apiKey": geoapify_key,
    "limit" : 20,
    "filter" : f"circle: {-176.5597},{-43.9535}, {10000}",
    "bias" : f"proximity: {-176.5597},{-43.9535}"
}

In [14]:
resp = requests.get("https://api.geoapify.com/v2/places", params=params)

In [15]:
resp

<Response [400]>

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accomodation.hotel",
    "apiKey": geoapify_key,
    "limit" : 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = row["Lng"]
    latitiude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {longitude},{latitiude}, {radius}"
    params["bias"] = f"proximity: {longitude},{latitiude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

In [53]:
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":10
}

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [54]:
params["filter"] = "Longitude"

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)